Here I explore adding variance of dense representation of words after removing stopwords. I also change the structre of the model as to seprate parts of inputs that will connect directly to the first Dense layer rather than go through the Embedding layer which didn't make a lot of sence 

In [25]:
import pandas as pd
import numpy as np

In [26]:
train_data = pd.read_csv(r"..\NLP_Bootcamp\Train_Dataset.csv")
test_data = pd.read_csv(r"..\NLP_Bootcamp\Test_Dataset.csv")

In [27]:
train_data.head()

,headline,is_sarcastic
0,supreme court votes 7-2 to legalize all worldl...,1
1,hungover man horrified to learn he made dozens...,1
2,emily's list founder: women are the 'problem s...,0
3,send your kids back to school with confidence,0
4,watch: experts talk pesticides and health,0


In [28]:
train_data["is_sarcastic"].value_counts()

0    23958
1    20304
Name: is_sarcastic, dtype: int64

almost balanced training dataset

## Data preprocessing

removing contractions: emily's -> emily is

In [29]:
import contractions
train_data["headline"] = train_data["headline"].apply(contractions.fix)

test_data["headline"] = test_data["headline"].apply(contractions.fix)

Removing Special Characters and Symbols

In [30]:
import re

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, ' ', text)
    return text

In [31]:
train_data["headline"] = train_data["headline"].apply(remove_special_characters)

test_data["headline"] = test_data["headline"].apply(remove_special_characters)

In [32]:
train_data.head()

,headline,is_sarcastic
0,supreme court votes 7 2 to legalize all worldl...,1
1,hungover man horrified to learn he made dozens...,1
2,emily s list founder women are the problem s...,0
3,send your kids back to school with confidence,0
4,watch experts talk pesticides and health,0


## Building the model

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data[["headline"]], train_data["is_sarcastic"], test_size=0.2, shuffle= False)

----------------------------------------------------------------------------------------------------------------------------------------------------------
#### Simple model using ML

In [35]:
import textblob

In [36]:
#creating statistical features and only choosing relevant ones
import string

X_train['char_count'] = X_train['headline'].apply(len)
X_train['word_count'] = X_train['headline'].apply(lambda x: len(x.split()))
#X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
#x_train_snt_obj = X_train['headline'].apply(lambda row: textblob.TextBlob(row).sentiment)
#X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
#X_train['Subjectivity'] = [obj.subjectivity for obj in x_train_snt_obj.values]

max_char_count = np.max(X_train['char_count'])
max_word_count = np.max(X_train['word_count'])

X_train['char_count'] = X_train['char_count'] / max_char_count
X_train['word_count'] = X_train['word_count'] / max_word_count




X_val['char_count'] = X_val['headline'].apply(len)
X_val['word_count'] = X_val['headline'].apply(lambda x: len(x.split()))
X_val['char_count'] = X_val['char_count'] / max_char_count
X_val['word_count'] = X_val['word_count'] / max_word_count

#X_val['word_density'] = X_val['char_count'] / (X_val['word_count']+1)
#X_val_snt_obj = X_val['headline'].apply(lambda row: textblob.TextBlob(row).sentiment)
#X_val['Polarity'] = [obj.polarity for obj in X_val_snt_obj.values]
#X_val['Subjectivity'] = [obj.subjectivity for obj in X_val_snt_obj.values]

In [37]:
X_train.head()

,headline,char_count,word_count
0,supreme court votes 7 2 to legalize all worldl...,0.057235,0.065789
1,hungover man horrified to learn he made dozens...,0.071274,0.078947
2,emily s list founder women are the problem s...,0.070194,0.072368
3,send your kids back to school with confidence,0.048596,0.052632
4,watch experts talk pesticides and health,0.044276,0.039474


In [38]:
#loading glove pretrained vectors

path_to_glove_file = r"..\glove.6B\glove.6B.300d.txt"

embeddings_index = {}
with open(path_to_glove_file, encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [39]:
#constructing new embedding features
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

t = Tokenizer()
t.fit_on_texts(X_train["headline"])

encoded_train = t.texts_to_sequences(X_train["headline"])
encoded_val = t.texts_to_sequences(X_val["headline"])

max_length = len(max(encoded_train, key= lambda x: len(x)))

padded_train = pad_sequences(encoded_train,
                             maxlen = max_length,
                             padding = "post")

padded_val = pad_sequences(encoded_val,
                           maxlen = max_length,
                           padding = "post")

print(padded_train.shape, padded_val.shape, type(padded_train))

vocab_size = len(t.word_index) + 1
vocab_size

(35409, 152) (8853, 152) <class 'numpy.ndarray'>


24565

In [40]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\33752\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\33752\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
X_train["processed_headline"] = X_train["headline"].apply(remove_special_characters, remove_digits= True)
X_train["processed_headline"] = X_train["processed_headline"].apply(remove_stopwords)


X_val["processed_headline"] = X_val["headline"].apply(remove_special_characters, remove_digits= True)
X_val["processed_headline"] = X_val["processed_headline"].apply(remove_stopwords)
X_train.head()

,headline,char_count,word_count,processed_headline
0,supreme court votes 7 2 to legalize all worldl...,0.057235,0.065789,supreme court votes legalize worldly vices
1,hungover man horrified to learn he made dozens...,0.071274,0.078947,hungover man horrified learn made dozens plans...
2,emily s list founder women are the problem s...,0.070194,0.072368,emily list founder women problem solvers congress
3,send your kids back to school with confidence,0.048596,0.052632,send kids back school confidence
4,watch experts talk pesticides and health,0.044276,0.039474,watch experts talk pesticides health


In [42]:
def headline_variance(text):
    epsilon = 0.0000001
    tokens = text.split(" ")
    vectors = np.array([np.array(embeddings_index[token], dtype= np.float32) for token in tokens if token in embeddings_index.keys()])
    if len(vectors) <= 1:
        return epsilon
    return np.var(vectors)

X_train["variance"] = X_train["processed_headline"].apply(headline_variance)

max_variance = np.max(X_train["variance"])
X_train["variance"] = X_train["variance"] / max_variance

X_val["variance"] = X_val["processed_headline"].apply(headline_variance)
X_val["variance"] = X_val["variance"] / max_variance

X_train.head()

,headline,char_count,word_count,processed_headline,variance
0,supreme court votes 7 2 to legalize all worldl...,0.057235,0.065789,supreme court votes legalize worldly vices,0.624866
1,hungover man horrified to learn he made dozens...,0.071274,0.078947,hungover man horrified learn made dozens plans...,0.425083
2,emily s list founder women are the problem s...,0.070194,0.072368,emily list founder women problem solvers congress,0.538932
3,send your kids back to school with confidence,0.048596,0.052632,send kids back school confidence,0.511886
4,watch experts talk pesticides and health,0.044276,0.039474,watch experts talk pesticides health,0.499833


In [43]:
X_train.isnull().sum()

headline              0
char_count            0
word_count            0
processed_headline    0
variance              0
dtype: int64

In [44]:
dense_vector_length = len(next(iter(embeddings_index.values())))
embedding_matrix = np.zeros((vocab_size, dense_vector_length)) # vector len of each word is 300

for word, i in t.word_index.items():
    if word in embeddings_index.keys():
        vec = embeddings_index[word]
        embedding_matrix[i] = vec

embedding_matrix.shape

(24565, 300)

In [45]:
#Merging the stastical features with embedding ones

X_train_comb = np.concatenate((X_train.drop(["headline", "processed_headline"], axis=1), padded_train), axis=1)
X_val_comb = np.concatenate((X_val.drop(["headline", "processed_headline"], axis=1), padded_val), axis=1)
print(X_train_comb.shape)
print(X_val_comb.shape)

(35409, 155)
(8853, 155)


In [46]:
#setting up callbacks for the model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


def checkpoint_path():
    return "./model_extra_features/weights.{epoch:02d}-{val_accuracy:.4f}.hdf5"

def log_dir():
    return "./logs/fit/" + datetime.now().strftime("%Y-%m-%d-%H:%M:%S")

earlystop = EarlyStopping(monitor = "val_accuracy", 
                          patience = 5, 
                          verbose = 1,  
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = "val_accuracy", 
                              factor = .3,
                              patience = 3,
                              verbose = 1, 
                              min_delta = 0.001)

In [47]:
#building the model
from tensorflow.keras import layers
from tensorflow.keras import Input, Model


input_layer = Input(shape = (X_train_comb.shape[1], ), name="input")

embedding = layers.Embedding(input_dim = vocab_size, 
                      output_dim = dense_vector_length, # glove vector size
                      weights = [embedding_matrix], 
                      trainable = False)(input_layer[:,3:])


flatten = layers.Flatten()(embedding)

merge = layers.concatenate([input_layer[:,:3], flatten], axis= 1)

dense = layers.Dense(256, activation = None, 
              kernel_initializer = "he_uniform")(merge)

dropout = layers.Dropout(.35)(dense)
activation = layers.Activation("relu")(dropout)

dense2 = layers.Dense(128, activation = 'relu')(activation)
dropout2 = layers.Dropout(0.35)(dense2)

dense3 = layers.Dense(32, activation = 'relu')(dropout2)

dense4 = layers.Dense(16, activation= 'relu')(dense3)

output = layers.Dense(1, activation = "sigmoid")(dense4)

model = Model(inputs = input_layer, outputs = output)

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 155)]        0           []                               
                                                                                                  
 tf.__operators__.getitem_2 (Sl  (None, 152)         0           ['input[0][0]']                  
 icingOpLambda)                                                                                   
                                                                                                  
 embedding_1 (Embedding)        (None, 152, 300)     7369500     ['tf.__operators__.getitem_2[0][0
                                                                 ]']                              
                                                                                            

In [48]:
#training the model
checkpoint = ModelCheckpoint(filepath = checkpoint_path(), 
                             monitor='val_accuracy', 
                             verbose = 1, 
                             save_best_only = True, 
                             mode = "max")

callbacks_list = [checkpoint, earlystop, reduce_lr]

history = model.fit(X_train_comb, Y_train, 
                    validation_data = (X_val_comb, Y_val), 
                    epochs = 30, 
                    batch_size = 64, 
                    callbacks = callbacks_list)

Epoch 1/30
554/554 [==============================] - ETA: 0s - loss: 0.4474 - accuracy: 0.7864
Epoch 00001: val_accuracy improved from -inf to 0.83938, saving model to ./model_extra_features\weights.01-0.8394.hdf5
554/554 [==============================] - 25s 45ms/step - loss: 0.4474 - accuracy: 0.7864 - val_loss: 0.3663 - val_accuracy: 0.8394 - lr: 0.0010
Epoch 2/30
553/554 [============================>.] - ETA: 0s - loss: 0.2649 - accuracy: 0.8938
Epoch 00002: val_accuracy improved from 0.83938 to 0.90037, saving model to ./model_extra_features\weights.02-0.9004.hdf5
554/554 [==============================] - 25s 46ms/step - loss: 0.2651 - accuracy: 0.8938 - val_loss: 0.2564 - val_accuracy: 0.9004 - lr: 0.0010
Epoch 3/30
553/554 [============================>.] - ETA: 0s - loss: 0.1438 - accuracy: 0.9455
Epoch 00003: val_accuracy improved from 0.90037 to 0.92025, saving model to ./model_extra_features\weights.03-0.9203.hdf5
554/554 [==============================] - 25s 45ms/step 

In [49]:
#valuating the model
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(X_val_comb)

predictions = [1 if x > 0.5 else 0 for x in predictions]

print(classification_report(Y_val, predictions))
pd.DataFrame(confusion_matrix(Y_val, predictions))


              precision    recall  f1-score   support

           0       0.93      0.94      0.94      4765
           1       0.93      0.92      0.93      4088

    accuracy                           0.93      8853
   macro avg       0.93      0.93      0.93      8853
weighted avg       0.93      0.93      0.93      8853



,0,1
0,4502,263
1,333,3755


In [50]:
#now we predict on our test_data so we start by applying the same transformaions we used on the training data
X_test = test_data
X_test['char_count'] = X_test['headline'].apply(len)
X_test['word_count'] = X_test['headline'].apply(lambda x: len(x.split()))

X_test['char_count'] = X_test['char_count'] / max_char_count
X_test['word_count'] = X_test['word_count'] / max_word_count

X_test["processed_headline"] = X_test["headline"].apply(remove_special_characters, remove_digits= True)
X_test["processed_headline"] = X_test["processed_headline"].apply(remove_stopwords)

X_test["variance"] = X_test["processed_headline"].apply(headline_variance)
X_test["variance"] = X_test["variance"] / max_variance



encoded_test = t.texts_to_sequences(X_test["headline"])

padded_test = pad_sequences(encoded_test,
                           maxlen = max_length,
                           padding = "post")

X_test_comb = np.concatenate((X_test.drop(["headline", "processed_headline"], axis=1), padded_test), axis=1)



result = model.predict(X_test_comb)
result = [1 if x > 0.5 else 0 for x in result]

In [51]:
#get time to not override different saves
from datetime import datetime

now = datetime.now()

dt_string = now.strftime("%d_%m_%Y_%H.%M.%S")

#save results localy
res = pd.DataFrame(result)

res.columns = ['prediction']
res.to_csv(f"prediction_results_{dt_string}.csv", index = False) 
print("file name= ", f"prediction_results_{dt_string}.csv")

file name=  prediction_results_12_12_2021_11.48.13.csv


It could be interesting to try and add the sum of distances between words in a headline as a feature after removing stop words like [to, if, and] etc. as generally in sarcastic headline we find words that usualy don't go together